In [1]:
import ipywidgets as widgets
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime
import pandas as pd
import time
import os
from re import search
from tqdm import tqdm
import plotly.express as px
import pyodbc 
import numpy as np
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
%matplotlib inline


In [2]:
#########################################################

################# Enter Values Here #####################

#########################################################
zipcode = 78741
pages = 1500
data_name = "new3"
#########################################################

In [3]:
# https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html

widget_car_name = widgets.Select(
    options=['Audi', 'BMW', 'Cadillac', 'Chrysler', 'Chevrolet', 'Dodge', 'Ferrari', 'Fiat', 'Ford', 'GMC', 'Honda', 'Hyundai', 'INFINITI', 'Jeep', 'Kia', 'Lamborghini', 'Land_Rover', 'Lincoln', 'Maserati', 'Mazda', 'Mercedes', 'Nissan', 'Porsche', 'Rolls', 'Subaru', 'Tesla', 'Toyota', 'Volkswagen', 'Volvo'], 
    value='Audi',
    # rows=10,
    description='Selecione o carro:',
    disabled=False
)
widget_car_name



Select(description='Selecione o carro:', options=('Audi', 'BMW', 'Cadillac', 'Chrysler', 'Chevrolet', 'Dodge',…

In [4]:
# Selecting car search data:

car_name = widget_car_name.value

In [5]:
# Selecting car search:

if car_name == 'Fiat':
    id_website_car = 'm98'
elif car_name == 'Toyota':
    id_website_car = 'm7'
elif car_name == 'Volkswagen':
    id_website_car = 'm55'
elif car_name == 'Mercedes':
    id_website_car = 'm43'
elif car_name == 'Porsche':
    id_website_car = 'm48'
elif car_name == 'Tesla':
    id_website_car = 'm112'
elif car_name == 'Land_Rover':
    id_website_car = 'm35'
elif car_name == 'Honda':
    id_website_car = 'm6'
elif car_name == 'Ford':
    id_website_car = 'm2'
elif car_name == 'Audi':
    id_website_car  = 'm19'
elif car_name == 'Ferrari':
    id_website_car  = 'm25'
elif car_name == 'Lamborghini':
    id_website_car = 'm34'
elif car_name == 'BMW':
    id_website_car = 'm3'
elif car_name == 'Chevrolet':
    id_website_car = 'm1'
elif car_name == 'Volvo':
    id_website_car = 'm56'
elif car_name == 'Kia':
    id_website_car = 'm33'
elif car_name == 'Hyundai':
    id_website_car = 'm28'    
elif car_name == 'Rolls':
    id_website_car = 'm49'    
elif car_name == 'Maserati':
    id_website_car = 'm40'
elif car_name == 'Subaru':
    id_website_car = 'm53'
elif car_name == 'Mazda':
    id_website_car = 'm42'
elif car_name == 'Cadillac':
    id_website_car = 'm22'
elif car_name == 'Jeep':
    id_website_car = 'm32'
elif car_name == 'Lincoln':
    id_website_car = 'm38'
elif car_name == 'Nissan':
    id_website_car = 'm12'
elif car_name == 'INFINITI':
    id_website_car = 'm84'
elif car_name == 'Dodge':
    id_website_car = 'm24'
elif car_name == 'Chrysler':
    id_website_car = 'm23'
elif car_name == 'GMC':
    id_website_car = 'm26'
    
zipcode = '{}'
link = "https://www.cargurus.com/Cars/inventorylisting/viewDetailsFilterViewInventoryListing.action?zip={}&minMileage=1&minPrice=2000&distance=50000&entitySelectingHelper.selectedEntity={}".format(zipcode, id_website_car)

raw_data = "_data/_{}_raw.csv".format(data_name)
clean_data = "_data/_{}_clean.csv".format(data_name)

print("\n ** ready to extract data from: {}...{}".format(link[:20], link[-20:]))
print("\n ** pages processing: {}".format(pages))


 ** ready to extract data from: https://www.cargurus...er.selectedEntity=m7

 ** pages processing: 1500


In [7]:
data = []
chromedriver = "chromedriver.exe"
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)
driver.get(link)
print("\n Select your filter. The countdown will start...")
print("\n 10, 9, 8, 7...")
time.sleep(13)
print("\n 6, 5, 4, 3, 2...")
time.sleep(13)
print("\n 1...")
time.sleep(13)
print("\n Process started!")
assert "CarGurus" in driver.title

while True:
    try:
        
        for i in tqdm(range(pages)):
            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")
            cars = soup.find_all("div", {"class":"MOfIEd XcutUU prRsnF"})

            if cars != []: # If doesn't exist information on the page, Click on final page.
                for car in cars:
                        if search("/mo", car.find_all("span")[2].text): # Insert only the info that has string ""/mo".
                            row = {}

                            # Finding Make
                            make = car.find_all("span")[1].text
                            make = make.split(" ", 5)
                            make = make[5]
                            make = make.split(" ", 1)
                            row["Make"] =  make[0]
                            
                            # Finding Model
                            model = car.find_all("span")[1].text
                            model = model.split(" ", 5)
                            row["Model"] = model[5]

                            # Finding Year
                            year = car.find_all("span")[1].text
                            year = year.split(" ")
                            row["Year"] = year[4]

                            # Finding Price    
                            price = car.find_all("span")[2].text
                            price = price.split(" ", 1)
                            price = price[0]
                            price = price.replace("$", "").replace(",", "")
                            row["Price"] = price

                            # Finding Mileage
                            mileage = car.find_all("span")[9].text
                            mileage = mileage.split(" ", 1)
                            mileage = mileage[0]
                            mileage = mileage.replace("$", "").replace(",", "")
                            row["Mileage"] = mileage

                            # Finding City
                            city = car.find_all("span")[11].text
                            city = city.replace(",", " -")
                            row["City"] = city

                            # Finding State
                            state = car.find_all("span")[11].text
                            row["State"] = state[-2:]
                            
                            # Month Analyzed
                            currentMonth = datetime.now()
                            row["History_Month_Listed"] = currentMonth.strftime("%B")

                            data.append(row)

                # Click on Next Page            
                next_page = driver.find_element(By.XPATH, '//span[text()="Next page"]')
                next_page.click()
                assert "CarGurus" in driver.title
                
            else:
                next_page = driver.find_element(By.XPATH, '//span[text()="Final page"]')
                next_page.click()
                assert "CarGurus" in driver.title

    except:
        break
            
#driver.close()
df = pd.DataFrame(data)
df = df.drop_duplicates()
df.to_csv(raw_data, encoding="ascii")
print("\n ** data extraction success!")
print("\n ** raw data added: {}".format(raw_data))

# Filter: >= 2018
# NationWide
# Less than 80k miles
# No accidents
# Hide Vehicles without photos
# 4+ Stars Customer Rating

C:\Users\Vini\AppData\Local\Temp\ipykernel_39300\3248290265.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver)



 Select your filter. The countdown will start...

 10, 9, 8, 7...

 6, 5, 4, 3, 2...

 1...

 Process started!


  0%|▎                                                                                | 5/1500 [00:07<34:55,  1.40s/it]


 ** data extraction success!

 ** raw data added: _data/_new3_raw.csv
